<a href="https://colab.research.google.com/github/Faycal99/Recommendation_System_For_Sports/blob/main/RecommendationSystemFirstTry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Recommendation System for sports based on differnt parameters including age,weight heigh, gender....etc

L'Objectif d’un système de recommandation hybride :

L’objectif principal de ce system est de fournir à chaque utilisateur des recommandations personnalisées (ici des programmes de fitness, régimes, ou équipements) en combinant intelligemment plusieurs approches pour maximiser la pertinence, même en présence de données incomplètes ou hétérogènes en utilisant le dataset de kaggle :https://www.kaggle.com/datasets/hossamfarhoud/gym-recommendation/code.

In [105]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
file_path = '/content/sample_data/gym recommendation.xlsx'
df = pd.read_excel(file_path)
pd.set_option('display.max_colwidth', 100)  # Adjust to desired width


In [106]:
df.head()

,ID,Sex,Age,Height,Weight,Hypertension,Diabetes,BMI,Level,Fitness Goal,Fitness Type,Exercises,Equipment,Diet,Recommendation
0,1,Male,18,1.68,47.5,No,No,16.83,Underweight,Weight Gain,Muscular Fitness,"Squats, deadlifts, bench presses, and overhead presses",Dumbbells and barbells,"Vegetables: (Carrots, Sweet Potato, and Lettuce); Protein Intake: (Red meats, poultry, fish, egg...",Follow a regular exercise schedule. Adhere to the exercise and diet plan to get better results. ...
1,2,Male,18,1.68,47.5,Yes,No,16.83,Underweight,Weight Gain,Muscular Fitness,"Squats, deadlifts, bench presses, and overhead presses","Light athletic shoes, resistance bands, and light dumbbells.","Vegetables: (Tomatoes, Garlic, leafy greens, broccoli, carrots, and bell peppers); Protein Intak...",Follow a regular exercise schedule. Adhere to the exercise and diet plan to get better results. ...
2,3,Male,18,1.68,47.5,No,Yes,16.83,Underweight,Weight Gain,Muscular Fitness,"Squats, yoga, deadlifts, bench presses, and overhead presses","Dumbbells, barbells and Blood glucose monitor","Vegetables: (Garlic, Roma Tomatoes, Capers and Iceberg Lettuce); Protein Intake: (Cheese Standwi...",Follow a regular exercise schedule. Adhere to the exercise and diet plan to get better results. ...
3,4,Male,18,1.68,47.5,Yes,Yes,16.83,Underweight,Weight Gain,Muscular Fitness,"Squats, yoga, deadlifts, bench presses, and overhead presses","Light athletic shoes, resistance bands, light dumbbells and a Blood glucose monitor.","Vegetables: (Garlic, Roma Tomatoes, Capers, Green Papper, and Iceberg Lettuce); Protein Intake: ...",Follow a regular exercise schedule. Adhere to the exercise and diet plan to get better results. ...
4,5,Male,18,1.68,47.5,No,No,16.83,Underweight,Weight Gain,Muscular Fitness,"Squats, deadlifts, bench presses, and overhead presses",Dumbbells and barbells,"Vegetables: (Carrots, Sweet Potato, Lettuce); Protein Intake: (Red meats, poultry, fish, eggs, d...",Follow a regular exercise schedule. Adhere to the exercise and diet plan to get better results. ...


Feauture normalization and cleaning we have those columns [Sex 	Age 	Height 	Weight 	Hypertension 	Diabetes 	BMI 	Level 	Fitness Goal 	Fitness Type 	Exercises 	Equipment 	Diet 	Recommendation] we are gonna consider as the features that we are going to use for our content based filter while Exercises 	Equipment 	Diet 	Recommendation for the recommendation

In [107]:
df = df.drop(columns=['BMI', 'Level'])

# Encode categorical values
label_encoder = LabelEncoder()
categorical_cols = ['Sex', 'Hypertension', 'Diabetes', 'Fitness Goal', 'Fitness Type']
for col in categorical_cols:
    df[col] = label_encoder.fit_transform(df[col])

In [108]:
df['Fitness Type'].value_counts(normalize=True)

,proportion
Fitness Type,
0,0.519638
1,0.480362


In [ ]:
df['Diabetes'].value_counts(normalize=True)

,proportion
Diabetes,
0,0.537254
1,0.462746


Normalization of age height and weight to be between -1 and 1 with the statndardScaler function

In [ ]:
# Normalize numerical columns
scaler = StandardScaler()
df[['Age', 'Height', 'Weight']] = scaler.fit_transform(df[['Age', 'Height', 'Weight']])

# Prepare input matrix X and separate recommendations
X = df.drop(columns=['ID', 'Exercises', 'Equipment', 'Diet', 'Recommendation'])

Here we split our dataset between train set and test

In [ ]:
X_train, X_test = train_test_split(X, test_size=0.1, random_state=42)
train_idx = X_train.index
test_idx = X_test.index

We add the clustring with kmeans in order to group our training set into 4 clusters

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=42)
df['cluster'] = kmeans.fit_predict(X)

In [ ]:
train_clusters = df.loc[train_idx, 'cluster']
test_clusters = df.loc[test_idx, 'cluster']

In [ ]:
# Content-based recommendation within the same cluster only
def content_based_clustered(test_row_pos, top_n=5):
    test_user_vector = X_test.iloc[[test_row_pos]]
    test_user_index = X_test.index[test_row_pos]
    test_user_cluster = df.loc[test_user_index, 'cluster']

    # Filter train users within the same cluster
    cluster_train_indices = train_idx[train_clusters == test_user_cluster]
    cluster_train_data = X_train.loc[cluster_train_indices]

    # Compute similarity
    similarity = cosine_similarity(test_user_vector, cluster_train_data)
    top_indices = similarity[0].argsort()[::-1][:top_n]
    similar_users = cluster_train_indices[top_indices]

    # Return recommendations of those similar users
    return df.loc[similar_users, ['Exercises', 'Diet', 'Equipment', 'Recommendation']]

# Example usage
result = content_based_clustered(test_row_pos=1, top_n=1)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
print("Top first recommendations from users in the same cluster:\n")
print(result)


🔍 Top 5 recommendations from users in the same cluster:

                                                   Exercises                                                                                                                                                                                                                     Diet               Equipment                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Combine the text fields for recommendations
df['combined_items'] = df['Exercises'].astype(str) + ' | ' + df['Diet'].astype(str) + ' | ' + df['Equipment'].astype(str)

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['combined_items'])

# Split the tfidf matrix into train and test to align with X_train and X_test
tfidf_train = tfidf_matrix[train_idx]
tfidf_test = tfidf_matrix[test_idx]

In [ ]:
import numpy as np

def evaluate_recommendations(test_row_pos, top_n=5):
    # Get test user index and their true combined items vector
    test_idx_val = test_idx[test_row_pos]
    true_vector = tfidf_test[test_row_pos]

    # Get top similar users by content-based filtering (profile similarity)
    test_user_vector = X_test.iloc[[test_row_pos]]
    similarity = cosine_similarity(test_user_vector, X_train)
    top_indices = similarity[0].argsort()[::-1][:top_n]
    similar_users_idx = train_idx[top_indices]

    # Aggregate recommended users' combined item vectors
    recommended_vectors = tfidf_matrix[similar_users_idx]

    # Compute similarity between true user and aggregated recommendations
    # One way: average cosine similarity between true user and each recommended user
    sims = cosine_similarity(true_vector, recommended_vectors)

    # Mean similarity score
    mean_sim = np.mean(sims)

    return mean_sim

# Example usage
score = evaluate_recommendations(test_row_pos=1, top_n=5)
print(f"Average similarity score of recommended items: {score:.4f}")

Average similarity score of recommended items: 1.0000


Now we give one test user,and find the top N most similar users in the training set using content-based similarity based on feautures X

In [ ]:
scores = []
for i in range(len(X_test)):
    score = evaluate_recommendations(i, top_n=5)
    scores.append(score)

print(f"Mean recommendation similarity over test set: {np.mean(scores):.4f}")

Mean recommendation similarity over test set: 0.9355


In [ ]:

# Seed random feedback between users (user-user matrix)
n_users = len(df)
feedback_matrix = pd.DataFrame(index=df.index, columns=df.index)
# Seed random feedback between users (vectorized version)
np.random.seed(42)
n_users = len(df)
feedback_array = np.random.choice([1, 0, -1], size=(n_users, n_users), p=[0.4, 0.4, 0.2])

# Set self-feedback to 1 on the diagonal
np.fill_diagonal(feedback_array, 1)

# Convert to DataFrame with same indexing as df
feedback_matrix = pd.DataFrame(feedback_array, index=df.index, columns=df.index)


# Collaborative filtering recommendation using feedback similarity
def cf_clustered_recommendation(test_row_pos, top_n=5):
    test_user_index = X_test.index[test_row_pos]
    test_user_cluster = df.loc[test_user_index, 'cluster']

    # Get users in the same cluster (excluding test user)
    cluster_users = df[df['cluster'] == test_user_cluster].index
    cluster_users = cluster_users.drop(test_user_index, errors='ignore')

    # Filter feedback matrix to the cluster only
    cluster_feedback = feedback_matrix.loc[test_user_index, cluster_users]

    # Keep only valid feedback (exclude -1)
    valid_feedback = cluster_feedback[cluster_feedback != -1]

    if valid_feedback.empty:
        return pd.DataFrame([{'Exercises': 'No match', 'Diet': '-', 'Equipment': '-', 'Recommendation': '-'}])

    # Sort by relevance score (CF similarity here is binary)
    top_users = valid_feedback.sort_values(ascending=False).head(top_n).index

    return df.loc[top_users, ['Exercises', 'Diet', 'Equipment', 'Recommendation']]

# Example usage
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
result = cf_clustered_recommendation(test_row_pos=1, top_n=5)
print("Top 5 collaborative recommendations based on user feedback:\n")
print(result)


🤝 Top 5 collaborative recommendations based on user feedback:

                                                          Exercises                                                                                                                                                                                                                                                          Diet                                                                             Equipment                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [ ]:
def evaluate_cf_model(top_n=1):
    total = 0
    relevant_hits = 0
    skipped = 0

    for test_row_pos in range(len(X_test)):
        test_user_index = X_test.index[test_row_pos]
        recommended = cf_clustered_recommendation(test_row_pos, top_n=top_n)

        if 'No match' in recommended.values:
            skipped += 1
            continue  # Skip users who had no valid recommendations

        # Evaluate: count how many of the recommended users have relevance == 1
        for idx in recommended.index:
            feedback = feedback_matrix.loc[test_user_index, idx]
            if feedback == 1:
                relevant_hits += 1
            total += 1

    if total == 0:
        return 0

    precision_at_k = relevant_hits / total
    coverage = (len(X_test) - skipped) / len(X_test)

    return {
        'Precision@k': precision_at_k,
        'Coverage': coverage,
        'Skipped': skipped,
        'Total Test Users': len(X_test)
    }

# 🔍 Evaluate the model
metrics = evaluate_cf_model(top_n=1)
print("\n📊 Evaluation Metrics:")
for k, v in metrics.items():
    print(f"{k}: {v}")



📊 Evaluation Metrics:
Precision@k: 1.0
Coverage: 1.0
Skipped: 0
Total Test Users: 1459


In [ ]:
def hybrid_recommendation(test_row_pos, top_n=1, alpha=0.5):
    cf_result = cf_clustered_recommendation(test_row_pos, top_n)
    cbf_result = content_based_clustered(test_row_pos, top_n)

    # If CF is unavailable, fallback to CBF only
    if cf_result is None:
        print("CF failed (no feedback), falling back to CBF only.\n")
        return cbf_result

    # If both are valid: combine by merging top_n/2 from each
    half_n = top_n // 2
    cf_top = cf_result.head(half_n)
    cbf_top = cbf_result.head(top_n - half_n)

    # Combine and drop duplicates
    combined = pd.concat([cf_top, cbf_top]).drop_duplicates().head(top_n)
    return combined

#  Example
test_row_pos = 1
result = hybrid_recommendation(test_row_pos=test_row_pos, top_n=1)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

print(f"\n Hybrid Recommendation for test user {X_test.index[test_row_pos]}:\n")
print(result)


🤝 Hybrid Recommendation for test user 10092:

                                                   Exercises                                                                                                                                                                                                                     Diet               Equipment                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [ ]:
def hybrid_recommendation(test_row_pos, top_n=5, alpha=0.5):
    cf_result = cf_clustered_recommendation(test_row_pos, top_n * 2)  # Get more candidates
    cbf_result = content_based_clustered(test_row_pos, top_n * 2)

    if cf_result is None and cbf_result is None:
        return pd.DataFrame([{'Exercises': 'No match', 'Diet': '-', 'Equipment': '-', 'Recommendation': '-'}])

    if cf_result is None:
        print(" CF failed (no feedback), fallback to CBF only.\n")
        return cbf_result.head(top_n)

    if cbf_result is None:
        print(" CBF failed, fallback to CF only.\n")
        return cf_result.head(top_n)

    # Assign scores (manual example)
    cf_result = cf_result.copy()
    cbf_result = cbf_result.copy()

    cf_result['score'] = [1 - (i / top_n) for i in range(len(cf_result))]
    cbf_result['score'] = [1 - (i / top_n) for i in range(len(cbf_result))]

    # Normalize weights
    cf_result['score'] *= alpha
    cbf_result['score'] *= (1 - alpha)

    combined = pd.concat([cf_result, cbf_result])
    combined = combined.groupby(['Exercises', 'Diet', 'Equipment', 'Recommendation'], as_index=False)['score'].sum()
    combined = combined.sort_values(by='score', ascending=False).head(top_n)

    return combined.drop(columns=['score'])

In [ ]:
hybrid_recommendation(test_row_pos=1, top_n=5, alpha=0.5)

,Exercises,Diet,Equipment,Recommendation
2,"Squats, deadlifts, bench presses, and overhead...","Vegetables: (Mixed greens, cherry tomatoes, cu...",Dumbbells and barbells,Follow a regular exercise schedule. Adhere to ...
1,"Squats, deadlifts, bench presses, and overhead...","Vegetables: (Garlic, mushroon, green papper an...","Light athletic shoes, resistance bands, and li...",Follow a regular exercise schedule. Adhere to ...
3,"Squats, yoga, deadlifts, bench presses, and ov...","Vegetables: (Garlic, Roma Tomatoes, Capers, Gr...","Light athletic shoes, resistance bands, light ...",Follow a regular exercise schedule. Adhere to ...
0,"Brisk walking, cycling, swimming, running , or...","Vegetables: (Broccoli, Carrots, Spinach, Lettu...","Ellipticals, Indoor Rowers,Treadmills, and Row...",Follow a regular exercise schedule. Adhere to ...
